In [1]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import yfinance as yf

In [3]:
for i in range(1):
    print(i)

0


In [2]:
#get data
data_tr = yf.download("AAPL", start="2017-01-01", end="2017-04-30")
data_te = yf.download("AAPL", start="2017-04-30", end="2018-01-30")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
#Environment
class Market_Env:
    
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        #.history records all 
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def step(self, act):
        reward = 0
        
        # act = 0: stay, 1: buy, 2: sell
        if act == 1:
            self.positions.append(self.data.iloc[self.t, :]['Close'])
        elif act == 2: # sell 
            if len(self.positions) == 0:
                #penalize for if we want to sell but there's nothing to sell
                reward = -1
            else:
                profits = 0
                #liquidate all currently held positions
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Close'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Close'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['Close'] - self.data.iloc[(self.t-1), :]['Close'])
        if (self.t==len(self.data)-1):
            self.done=True
        
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

In [4]:
env = Market_Env(data_tr)
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print (pact)
    print(env.step(pact))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.032501220703125], 0, False)
0
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.032501220703125, 0.14750099182128906], 0, False)
1
([0.3250007629394531, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
# Q Neural Network
class Q_Network(nn.Module):
        
    def __init__(self,obs_len,hidden_size,actions_n):
            
        super(Q_Network,self).__init__()
            
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
        
        
    def forward(self,x):
        h =  self.fc_val(x)
        return (h)

In [6]:
hidden_size=100
input_size=env.history_t+1
output_size=3
USE_CUDA = False
LR = 0.001

In [7]:
Q = Q_Network(input_size, hidden_size, output_size)

In [8]:
Q_ast = copy.deepcopy(Q)

In [9]:
if USE_CUDA:
    Q = Q.cuda()
loss_function = nn.MSELoss()
optimizer = optim.Adam(list(Q.parameters()), lr=LR)

In [10]:
#more params
epoch_num = 50
step_max = len(env.data)-1
memory_size = 200
batch_size = 50
gamma = 0.97

In [11]:
### ?
memory = []
total_step = 0
total_rewards = []
total_losses = []
epsilon = 1.0
epsilon_decrease = 1e-3
epsilon_min = 0.1
start_reduce_epsilon = 200
train_freq = 10
update_q_freq = 20
gamma = 0.97
show_log_freq = 5

In [12]:
start = time.time()
for epoch in range(epoch_num):

    pobs = env.reset()
    step = 0
    done = False
    total_reward = 0
    total_loss = 0

    while not done and step < step_max:

        # select act
        pact = np.random.randint(3)
        if np.random.rand() > epsilon:
            pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
            pact = np.argmax(pact.data)
            pact = pact.numpy()

        # act
        obs, reward, done = env.step(pact)

        # add memory
        memory.append((pobs, pact, reward, obs, done))
        if len(memory) > memory_size:
            memory.pop(0)

        # train or update q
        if len(memory) == memory_size:
            if total_step % train_freq == 0:
                shuffled_memory = np.random.permutation(memory)
                memory_idx = range(len(shuffled_memory))
                for i in memory_idx[::batch_size]:
                    batch = np.array(shuffled_memory[i:i+batch_size])
                    b_pobs = np.array(batch[:, 0].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_pact = np.array(batch[:, 1].tolist(), dtype=np.int32)
                    b_reward = np.array(batch[:, 2].tolist(), dtype=np.int32)
                    b_obs = np.array(batch[:, 3].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)

                    q = Q(torch.from_numpy(b_pobs))
                    q_ = Q_ast(torch.from_numpy(b_obs))
                    maxq = np.max(q_.data.numpy(),axis=1)
                    target = copy.deepcopy(q.data)
                    for j in range(batch_size):
                        target[j, b_pact[j]] = b_reward[j]+gamma*maxq[j]*(not b_done[j])
                    Q.zero_grad()
                    loss = loss_function(q, target)
                    total_loss += loss.data.item()
                    loss.backward()
                    optimizer.step()
                    
            if total_step % update_q_freq == 0:
                Q_ast = copy.deepcopy(Q)
                
            # epsilon
            if epsilon > epsilon_min and total_step > start_reduce_epsilon:
                epsilon -= epsilon_decrease

            # next step
            total_reward += reward
            pobs = obs
            step += 1
            total_step += 1

        total_rewards.append(total_reward)
        total_losses.append(total_loss)

        if (epoch+1) % show_log_freq == 0:
            log_reward = sum(total_rewards[((epoch+1)-show_log_freq):])/show_log_freq
            log_loss = sum(total_losses[((epoch+1)-show_log_freq):])/show_log_freq
            elapsed_time = time.time()-start
            print('\t'.join(map(str, [epoch+1, epsilon, total_step, log_reward, log_loss, elapsed_time])))
            start = time.time()
            
#return Q, total_losses, total_rewards

/home/m73/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<ipython-input-12-c12c7fee9efc>:38: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)


5	1.0	122	-163.4	31.71398105695844	0.2895996570587158
5	1.0	123	-163.6	31.71398105695844	0.00028514862060546875
5	1.0	124	-163.8	31.71398105695844	0.0002758502960205078
5	1.0	125	-164.0	31.71398105695844	0.0005221366882324219
5	1.0	126	-164.2	31.71398105695844	0.0006468296051025391
5	1.0	127	-164.4	31.71398105695844	0.0005216598510742188
5	1.0	128	-164.6	31.71398105695844	0.0005273818969726562
5	1.0	129	-164.6	31.71398105695844	0.0005393028259277344
5	1.0	130	-164.8	31.71398105695844	0.0002467632293701172
5	1.0	131	-165.0	31.780858839303257	0.008646011352539062
5	1.0	132	-165.4	31.847736621648075	0.00030517578125
5	1.0	133	-165.8	31.914614403992893	0.00027823448181152344
5	1.0	134	-166.2	31.98149218633771	0.00027489662170410156
5	1.0	135	-166.8	32.04836996868253	0.0002796649932861328
5	1.0	136	-167.4	32.11524775102735	0.0005223751068115234
5	1.0	137	-168.0	32.182125533372165	0.0006551742553710938
5	1.0	138	-168.6	32.24900331571698	0.000522613525390625
5	1.0	139	-169.2	32.3158810980618	

15	0.27899999999999936	922	-405.8	214.58532702624797	0.46454787254333496
15	0.27799999999999936	923	-405.8	214.58532702624797	0.0008647441864013672
15	0.27699999999999936	924	-405.6	214.58532702624797	0.0006654262542724609
15	0.27599999999999936	925	-405.4	214.58532702624797	0.0005044937133789062
15	0.27499999999999936	926	-405.0	214.58532702624797	0.0006124973297119141
15	0.27399999999999936	927	-404.6	214.58532702624797	0.0004394054412841797
15	0.27299999999999935	928	-404.2	214.58532702624797	0.0004749298095703125
15	0.27199999999999935	929	-403.8	214.58532702624797	0.0005805492401123047
15	0.27099999999999935	930	-403.4	214.58532702624797	0.0008504390716552734
15	0.26999999999999935	931	-402.8	214.71778302788735	0.008396148681640625
15	0.26899999999999935	932	-402.2	214.8502390295267	0.00047516822814941406
15	0.26799999999999935	933	-401.8	214.9826950311661	0.00043654441833496094
15	0.26699999999999935	934	-401.4	215.11515103280544	0.0004458427429199219
15	0.26599999999999935	935	-

25	0.0999999999999992	1722	379.2	476.4756493076682	0.47432875633239746
25	0.0999999999999992	1723	379.2	476.4756493076682	0.0009152889251708984
25	0.0999999999999992	1724	379.4	476.4756493076682	0.0007331371307373047
25	0.0999999999999992	1725	379.4	476.4756493076682	0.0004899501800537109
25	0.0999999999999992	1726	379.2	476.4756493076682	0.0004892349243164062
25	0.0999999999999992	1727	379.0	476.4756493076682	0.0007143020629882812
25	0.0999999999999992	1728	379.0	476.4756493076682	0.0006060600280761719
25	0.0999999999999992	1729	379.0	476.4756493076682	0.0007159709930419922
25	0.0999999999999992	1730	379.0	476.4756493076682	0.0008370876312255859
25	0.0999999999999992	1731	379.2	476.5244072698057	0.008234500885009766
25	0.0999999999999992	1732	379.4	476.5731652319431	0.0007414817810058594
25	0.0999999999999992	1733	379.6	476.6219231940806	0.0008318424224853516
25	0.0999999999999992	1734	379.8	476.67068115621805	0.0005068778991699219
25	0.0999999999999992	1735	380.0	476.7194391183555	0.

35	0.0999999999999992	2522	1170.2	827.6266552686691	0.7245616912841797
35	0.0999999999999992	2523	1170.2	827.6266552686691	0.0008730888366699219
35	0.0999999999999992	2524	1170.4	827.6266552686691	0.0006785392761230469
35	0.0999999999999992	2525	1170.6	827.6266552686691	0.0006728172302246094
35	0.0999999999999992	2526	1171.0	827.6266552686691	0.0005981922149658203
35	0.0999999999999992	2527	1171.4	827.6266552686691	0.0005204677581787109
35	0.0999999999999992	2528	1172.0	827.6266552686691	0.0006198883056640625
35	0.0999999999999992	2529	1172.6	827.6266552686691	0.0006844997406005859
35	0.0999999999999992	2530	1173.2	827.6266552686691	0.0007889270782470703
35	0.0999999999999992	2531	1174.0	827.703533422947	0.008433103561401367
35	0.0999999999999992	2532	1174.8	827.7804115772248	0.0003075599670410156
35	0.0999999999999992	2533	1175.6	827.8572897315025	0.0007681846618652344
35	0.0999999999999992	2534	1176.4	827.9341678857803	0.0008006095886230469
35	0.0999999999999992	2535	1177.2	828.01104

45	0.0999999999999992	3322	2753.0	934.9154254302382	0.4422323703765869
45	0.0999999999999992	3323	2753.0	934.9154254302382	0.0009138584136962891
45	0.0999999999999992	3324	2753.0	934.9154254302382	0.0006639957427978516
45	0.0999999999999992	3325	2753.0	934.9154254302382	0.0007317066192626953
45	0.0999999999999992	3326	2753.0	934.9154254302382	0.0007128715515136719
45	0.0999999999999992	3327	2753.0	934.9154254302382	0.0007123947143554688
45	0.0999999999999992	3328	2753.2	934.9154254302382	0.0006926059722900391
45	0.0999999999999992	3329	2753.4	934.9154254302382	0.0006871223449707031
45	0.0999999999999992	3330	2753.6	934.9154254302382	0.0007750988006591797
45	0.0999999999999992	3331	2753.8	934.9473556414247	0.008207321166992188
45	0.0999999999999992	3332	2754.0	934.979285852611	0.0007393360137939453
45	0.0999999999999992	3333	2754.2	935.0112160637975	0.0007143020629882812
45	0.0999999999999992	3334	2754.4	935.0431462749839	0.0006968975067138672
45	0.0999999999999992	3335	2754.6	935.07507

In [13]:
test_env = Market_Env(data_te)
pobs = test_env.reset()
test_acts = []
test_rewards = []

for _ in range(len(test_env.data)-1):
    
    pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
    pact = np.argmax(pact.data)
    test_acts.append(pact.item())
            
    obs, reward, done = test_env.step(pact.numpy())
    test_rewards.append(reward)

    pobs = obs
        
test_profits = test_env.profits

In [14]:
test_profits

4.867500305175781